# Hyperparameter Tuning using HyperDrive

Install and import dependencies

In [1]:
!pip install kaggle

If the import of data_prep fails, see the README on how to download and copy the kaggle.json

In [2]:
from azureml.core.workspace import Workspace
from azureml.core import Experiment, Model, Webservice
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
from azureml.train.hyperdrive.parameter_expressions import choice
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
import joblib

import data_prep

## Dataset

### Overview
The Portable Executable (PE) format is a file format for executables, object code, DLLs and others used in 32-bit and 64-bit versions of Windows operating systems. The header of PE files contains a number for things like the size of the file, imported libraries, and more. This dataset from [Kaggle](https://www.kaggle.com/divg07/malware-analysis-dataset) contains data extracted from PE headers from both known malware samples and benign software samples.

### Task
The task for this project is to train models to classify whether an executable is malware or benign using features extracted from their PE Header. The 'legitimate' column in the dataset is 1 when the executible file is from a legitimate source (aka benign software or goodware), and 0 when it is malware.  


In [3]:
ws = Workspace.from_config()
ws.write_config(path='./training')
experiment_name = 'hyperdrive_experiment'

experiment=Experiment(ws, experiment_name)

dataset = data_prep.get_dataset(ws)

## Find or Create Compute Cluster

In [4]:
cpu_cluster_name = "cpu-cluster"

# Check if the cluster exists. If there is an error, create the cluster
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2',
                                                            max_nodes=10)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Hyperdrive Configuration

In the custom training script, we define a Scikit Learn Random Forrest Classifier. There are 4 hyperparameters we are going to tune with Hyperdrive: 
* n_estimators: the number of trees in the Random forrest
* max_depth: the maximum depth of the trees in the forrest
* min_samples_split: the minimum number of samples required to split an internal node
* min_samples_leaf: the minimum number of samples required to be at a leaf node

In [5]:
# Early termination policy
early_termination_policy = BanditPolicy(slack_amount=0.2)

# Random Parameter sampling
param_sampling = RandomParameterSampling({
                    "n_estimators": choice(10,50,100,150,200),
                    "max_depth": choice(0,2,5,10),
                    "min_samples_split": choice(2,3,4,5),
                    "min_samples_leaf": choice(1,2,3,4,5)
                })

# SKLearn Estimator
estimator = SKLearn(source_directory="./training", 
                entry_script="train.py",
                compute_target=cpu_cluster)

# Hyperdrive Configuration
hyperdrive_run_config = HyperDriveConfig(param_sampling,
                                    primary_metric_name="Accuracy", 
                                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                    max_total_runs=50,
                                    estimator=estimator,
                                    policy=early_termination_policy)

In [6]:
hyperdrive_run = experiment.submit(hyperdrive_run_config)

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


## Run Details

In [7]:
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Best Model

In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [8]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])
print('\n Num Estimators:',parameter_values[7])
print('\n Max Depth:',parameter_values[1])
print('\n Min Samples Split:',parameter_values[5])
print('\n Min Samples Leaf:',parameter_values[3])

Best Run Id:  HD_145864a7-2448-49cd-a7c8-6aa24c9a7d3e_10

 Accuracy: 0.9994512248660988

 Num Estimators: 200

 Max Depth: 0

 Min Samples Split: 3

 Min Samples Leaf: 1


In [9]:
model = best_run.register_model(model_name = 'best_hyperdrive_model', model_path = 'outputs/model.joblib', model_framework=Model.Framework.SCIKITLEARN)